# Importing libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import (Embedding, Bidirectional, LSTM, Dense,Dropout, BatchNormalization, SpatialDropout1D , Input)
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.regularizers import l2
import re

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
print("imported libraries successfully")


2025-11-24 08:50:11.428390: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763974211.732362      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763974211.817329      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

imported libraries successfully


In [2]:
# Download required NLTK data
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)


# Explore data

In [3]:
fake= pd.read_csv("https://raw.githubusercontent.com/laxmimerit/fake-real-news-dataset/refs/heads/main/data/Fake.csv")


In [4]:
real= pd.read_csv("https://raw.githubusercontent.com/laxmimerit/fake-real-news-dataset/refs/heads/main/data/True.csv")


In [5]:
real.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [6]:
real.drop('date' , axis=1 , inplace=True)


In [7]:
real.drop('subject' , axis=1 , inplace=True)


In [8]:
fake.drop('date' , axis=1 , inplace=True)
fake.drop('subject' , axis=1 , inplace=True)


In [9]:
df = pd.concat([real , fake], ignore_index=True)

In [10]:
df

,title,text
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...
...,...,...
44893,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...
44894,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...
44895,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...
44896,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...


# Text preprocessing

In [ ]:
def preprocess_text(text):
    """
    Text preprocessing function 
    This function removes urls , html tags , special characters and extra white spaces and performs lemmetization
    
    """
    if pd.isna(text):
        return ""
    
    # Convert to lowercase
    text = text.lower()

    text = re.sub(r'\b(reuters|associated press|ap|upi|afp|bloomberg)\b', '', text)

    text = re.sub(r'^\s*[\w\s]+(?:\(.*\))?\s*-\s*', '', text)
    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Remove stopwords and lemmatize
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words and len(word) > 2]
    
    return ' '.join(words)



In [ ]:
# Apply preprocessing
df['cleaned_text'] = df['text'].apply(preprocess_text)


In [ ]:
df.head()

In [ ]:
df.drop('text' , axis=1 , inplace=True)


In [ ]:

# Remove empty texts after preprocessing
df = df[df['cleaned_text'].str.len() > 0].reset_index(drop=True)

In [ ]:
print("clenaed df shape" , df.shape)

# Prepare data for training

In [ ]:
voc_size=10000
max_len=300
embedding_dim=200

In [ ]:
x= df['cleaned_text']
y = df['label']
x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size=0.3, random_state=42, stratify=y)
# Validation split
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

In [ ]:
x

In [ ]:
print("Training set size :" , len(x_train))
print("Validation set size :" , len(x_val))
print("Test set size :" , len(x_test))

# Tokenization

In [ ]:
tokenizer= Tokenizer(num_words=voc_size , oov_token='<OOV>')
tokenizer.fit_on_texts(x_train)

# Convert text to sequences

In [ ]:
x_train_seq = tokenizer.texts_to_sequences(x_train)
x_val_seq = tokenizer.texts_to_sequences(x_val)
x_test_seq = tokenizer.texts_to_sequences(x_test)

# Padding

In [ ]:
xtrain_pad= pad_sequences(x_train_seq , maxlen=max_len , padding='post', truncating ='post')
xval_pad= pad_sequences(x_val_seq , maxlen=max_len , padding='post', truncating ='post')
xtest_pad= pad_sequences(x_test_seq , maxlen=max_len , padding='post', truncating ='post')

# Training Bi-LSTM model

In [ ]:
# Embedding layer
model= keras.Sequential()
model.add(Input(shape=(max_len, )))
model.add(Embedding(input_dim=voc_size ,output_dim=embedding_dim,mask_zero=True))
model.add(SpatialDropout1D(0.3))
model.add(Bidirectional(LSTM(128 ,return_sequences=True , dropout=0.3 , kernel_regularizer=l2(0.01))))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(64 , dropout=0.3 , kernel_regularizer=l2(0.01))))
model.add(BatchNormalization())
model.add(Dense(64 , activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(32 , activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))
print("done")

In [ ]:
optimizer=keras.optimizers.Adam(learning_rate=0.001)



In [ ]:
model.compile(optimizer=optimizer ,
             loss='binary_crossentropy',
             metrics=['accuracy', 
                     keras.metrics.Precision(name='precision'),
                     keras.metrics.Recall(name='recall'),
                     keras.metrics.AUC(name='auc'),
                     ])

In [ ]:
model.summary()

# Callbacks

In [ ]:
callbacks = [
    # Early stopping to prevent overfitting
    EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    
    # Reduce learning rate when plateau
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-7,
        verbose=1
    ),
    
    # Save best model
    ModelCheckpoint(
        'best_bilstm_model.keras',
        monitor='val_auc',
        mode='max',
        save_best_only=True,
        verbose=1
    )
]

In [ ]:
history= model.fit(xtrain_pad , y_train, batch_size=64, epochs=30 ,validation_data=(xval_pad , y_val), callbacks=callbacks , verbose=1 )

# visualize training history

In [ ]:
fig ,axes= plt.subplots(2,2 , figsize=(15,10))

#Accuracy
# Accuracy
axes[0, 0].plot(history.history['accuracy'], label='Train Accuracy')
axes[0, 0].plot(history.history['val_accuracy'], label='Val Accuracy')
axes[0, 0].set_title('Model Accuracy')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Accuracy')
axes[0, 0].legend()
axes[0, 0].grid(True)

# Loss
axes[0, 1].plot(history.history['loss'], label='Train Loss')
axes[0, 1].plot(history.history['val_loss'], label='Val Loss')
axes[0, 1].set_title('Model Loss')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].legend()
axes[0, 1].grid(True)

# AUC
axes[1, 0].plot(history.history['auc'], label='Train AUC')
axes[1, 0].plot(history.history['val_auc'], label='Val AUC')
axes[1, 0].set_title('Model AUC')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('AUC')
axes[1, 0].legend()
axes[1, 0].grid(True)

# Precision & Recall
axes[1, 1].plot(history.history['precision'], label='Train Precision')
axes[1, 1].plot(history.history['recall'], label='Train Recall')
axes[1, 1].plot(history.history['val_precision'], label='Val Precision')
axes[1, 1].plot(history.history['val_recall'], label='Val Recall')
axes[1, 1].set_title('Precision & Recall')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Score')
axes[1, 1].legend()
axes[1, 1].grid(True)
    
plt.tight_layout()
plt.savefig('training_history.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Predictions
y_pred_proba =model.predict(xtest_pad)
y_pred = (y_pred_proba > 0.5).astype(int)

# Classification report
print("\n=== CLASSIFICATION REPORT ===")
print(classification_report(y_test, y_pred, 
                            target_names=['Real', 'Fake'],
                            digits=4))

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Real', 'Fake'],
            yticklabels=['Real', 'Fake'])
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()
    

In [ ]:
# ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
auc_score = roc_auc_score(y_test, y_pred_proba)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'BiLSTM (AUC = {auc_score:.4f})')
plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.grid(True)
plt.savefig('roc_curve.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Save the model
model.save('fake_news_bilstm_final3.keras')

# Save tokenizer
import pickle
with open('tokenizer3.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

print("\n=== MODEL SAVED ===")
print("Model saved as: fake_news_bilstm_final.keras")
print("Tokenizer saved as: tokenizer.pkl")

In [ ]:
df.head()

In [ ]:
# Run this in your notebook to see a sample
print(df[df['label'] == 1].head(1)['cleaned_text'].values)

In [ ]:
# Check the content of label 1
print("SAMPLE OF LABEL 1 (REAL):")
print(df[df['label'] == 1]['cleaned_text'].iloc[0][:200]) 
# You should see "Reuters" or real news text here.

# Check the content of label 0
print("\nSAMPLE OF LABEL 0 (FAKE):")
print(df[df['label'] == 0]['cleaned_text'].iloc[0][:200])
# You should see conspiracy theories or fake headlines here.